<center><img src="http://alacip.org/wp-content/uploads/2014/03/logoEscalacip1.png" width="500"></center>


<center> <h1>Curso: Introducción al Python</h1> </center>

<br></br>

* Profesor:  <a href="http://www.pucp.edu.pe/profesor/jose-manuel-magallanes/" target="_blank">Dr. José Manuel Magallanes, PhD</a> ([jmagallanes@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe))<br>
    - Profesor del **Departamento de Ciencias Sociales, Pontificia Universidad Católica del Peru**.<br>
    - Senior Data Scientist del **eScience Institute** and Visiting Professor at **Evans School of Public Policy and Governance, University of Washington**.<br>
    - Fellow Catalyst, **Berkeley Initiative for Transparency in Social Sciences, UC Berkeley**.


## Parte 4:  Formateando Valores en Python

<a id='beginning'></a>

Una vez que los datos han sido limpiados, es momento de ver si tienen el formato para ser tratados estadísticamente, es decir, que representan el correcto tipo de [escala](#scale) y que representen una adecuada [monotonía](#monotony).

Traigamos unos datos conocidos, pero acelerando la limpieza:

In [14]:
wikiLink="https://en.wikipedia.org/wiki/List_of_freedom_indices" 

#traer tabla
import pandas as pd
DF=pd.read_html(wikiLink,header=0,flavor='bs4',attrs={'class': 'wikitable sortable',})[0]

#limpiando celdas
import re  
pattern='\\s+|\\d+|\\[|\\]'
nothing=''
DF.columns=[re.sub(pattern,nothing,element) for element in DF.columns]
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Abkhazia,partly free,NaN,NaN,NaN
1,Afghanistan,not free,mostly unfree,difficult situation,authoritarian regime
2,Albania,partly free,moderately free,noticeable problems,hybrid regime
3,Algeria,not free,repressed,difficult situation,authoritarian regime
4,Andorra,free,NaN,satisfactory situation,NaN


____
<a id='scale'></a>

### Las escalas de medición

Para saber si están en la escala correcta, debemos usar _dtypes_:

In [15]:
DF.dtypes

Country                   object
FreedomintheWorld         object
IndexofEconomicFreedom    object
PressFreedomIndex         object
DemocracyIndex            object
dtype: object

Los cuatro indices son categorías, no texto (_object_). Hagamos la conversión:

In [16]:
headers=DF.columns
DF[headers[1:]]=DF[headers[1:]].astype('category')

In [17]:
# sale:
DF.dtypes

Country                     object
FreedomintheWorld         category
IndexofEconomicFreedom    category
PressFreedomIndex         category
DemocracyIndex            category
dtype: object

Este cambio es imperceptible a la vista:

In [18]:
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Abkhazia,partly free,NaN,NaN,NaN
1,Afghanistan,not free,mostly unfree,difficult situation,authoritarian regime
2,Albania,partly free,moderately free,noticeable problems,hybrid regime
3,Algeria,not free,repressed,difficult situation,authoritarian regime
4,Andorra,free,NaN,satisfactory situation,NaN


Ahora si podemos saber sus valores:

In [19]:
DF.FreedomintheWorld.cat.categories

Index(['free', 'not free', 'partly free'], dtype='object')

In [20]:
DF.IndexofEconomicFreedom.cat.categories

Index(['free', 'moderately free', 'mostly free', 'mostly unfree', 'repressed'], dtype='object')

In [21]:
DF.PressFreedomIndex.cat.categories

Index(['difficult situation', 'good situation', 'noticeable problems',
       'satisfactory situation', 'very serious situation'],
      dtype='object')

In [22]:
DF.DemocracyIndex.cat.categories

Index(['authoritarian regime', 'flawed democracy', 'full democracy',
       'hybrid regime'],
      dtype='object')

Vemos que tenemos hasta 5 niveles en 2 variables, y 3 y 4 niveles en otras. De ahi que lo prudente es encontrar la distribución común de valores que refleja la ordinalidad, y los máximos y mínimos. 

Veamos como volverla ordinal. Primero los valores:

In [23]:
oldFree=list(DF.FreedomintheWorld.cat.categories)
newFree=['very good','very bad','middle']
recodeFree={old:new for old,new in zip (oldFree,newFree)}

oldEco=list(DF.IndexofEconomicFreedom.cat.categories)
newEco=['very good','middle','good','bad','very bad']
recodeEco={old:new for old,new in zip (oldEco,newEco)}


oldPress=list(DF.PressFreedomIndex.cat.categories)
newPress=['bad','very good','middle','good','very bad']
recodePress={old:new for old,new in zip (oldPress,newPress)}

oldDemo=list(DF.DemocracyIndex.cat.categories)
newDemo=['very bad','good','very good','bad']
recodeDemo={old:new for old,new in zip (oldDemo,newDemo)}

Ahora usamos los dicts creado para recodificar:

In [24]:
DF.FreedomintheWorld.cat.rename_categories(recodeFree,inplace=True)

DF.IndexofEconomicFreedom.cat.rename_categories(recodeEco,inplace=True)

DF.PressFreedomIndex.cat.rename_categories(recodePress,inplace=True)

DF.DemocracyIndex.cat.rename_categories(recodeDemo,inplace=True)

# veamos:
DF

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Abkhazia,middle,NaN,NaN,NaN
1,Afghanistan,very bad,bad,bad,very bad
2,Albania,middle,middle,middle,bad
3,Algeria,very bad,very bad,bad,very bad
4,Andorra,very good,NaN,good,NaN
5,Angola,very bad,very bad,bad,very bad
6,Antigua and Barbuda,very good,NaN,good,NaN
7,Argentina,very good,bad,middle,good
8,Armenia,middle,middle,middle,bad
9,Australia,very good,very good,good,very good


Los datos aun no son ordinales, pero aqui serán:

In [25]:
from pandas.api.types import CategoricalDtype


ordinal = CategoricalDtype(categories=['very good','good','middle','bad','very bad'],ordered=True)

to_Order=lambda x: x.astype(ordinal)
DF[headers[1:]]=DF[headers[1:]].apply(to_Order)

In [26]:
# asi va:
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Abkhazia,middle,NaN,NaN,NaN
1,Afghanistan,very bad,bad,bad,very bad
2,Albania,middle,middle,middle,bad
3,Algeria,very bad,very bad,bad,very bad
4,Andorra,very good,NaN,good,NaN


Notemos que las modalidades no usadas están presentes:

In [27]:
DF.FreedomintheWorld.value_counts(sort=False,dropna=False)

very good    89
good          0
middle       62
bad           0
very bad     55
NaN           1
Name: FreedomintheWorld, dtype: int64

Verificaciones adicionales:

In [28]:
#las categorias:
DF.PressFreedomIndex.cat.categories

Index(['very good', 'good', 'middle', 'bad', 'very bad'], dtype='object')

In [29]:
#tipo de escala?
DF.PressFreedomIndex.cat.ordered

True

[Go to page beginning](#beginning)
____
<a id='monotony'></a>

### Cambio de Monotonía:

Verifiquemos si está bien la asignación que hemos hecho:

In [30]:
DF.PressFreedomIndex.head()

0       NaN
1       bad
2    middle
3       bad
4      good
Name: PressFreedomIndex, dtype: category
Categories (5, object): [very good < good < middle < bad < very bad]

In [31]:
DF.PressFreedomIndex.max()

'very bad'

Este es un caso donde quiza la intensidad creciente debe ser hacia el sentido positivo del concepto. Claro que pudimos hacerlo al inicio, pero aprovechemos para saber cómo se hace.

Para ello crearé una función:

In [32]:
def changeMonotony(aColumn):
    # Invierto las categorias:
    newOrder= aColumn.cat.categories[::-1]  # [::-1]  reverses the list.
    # aplico función
    return aColumn.cat.reorder_categories(newOrder,ordered=True)

Esta función la aplica de nuevo, columna por columna:

In [33]:
# SOLO UNA VEZ!! (sino reintenta)
DF[headers[1:]]=DF[headers[1:]].apply(changeMonotony)

¿Funcionó?

In [34]:
DF.PressFreedomIndex.head(20)

0           NaN
1           bad
2        middle
3           bad
4          good
5           bad
6          good
7        middle
8        middle
9          good
10    very good
11     very bad
12         good
13     very bad
14          bad
15         good
16          bad
17    very good
18         good
19       middle
Name: PressFreedomIndex, dtype: category
Categories (5, object): [very bad < bad < middle < good < very good]

In [35]:
DF.PressFreedomIndex.max()

'very good'

Todo lo que hemos trabajado podríamos entregarselo a R para que haga su trabajo estadístico, pero como no tiene metadata, es mejor guardar los indices ordinales como número:

In [36]:
oldlevels=['very bad','bad','middle','good','very good']
newlevels=[1,2,3,4,5]
recodeLevels={old:new for old,new in zip (oldlevels,newlevels)}

In [37]:
renamer=lambda column: column.cat.rename_categories(recodeLevels)
DF[headers[1:]]=DF[headers[1:]].apply(renamer)
DF.head(10)

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Abkhazia,3,NaN,NaN,NaN
1,Afghanistan,1,2.0,2.0,1.0
2,Albania,3,3.0,3.0,2.0
3,Algeria,1,1.0,2.0,1.0
4,Andorra,5,NaN,4.0,NaN
5,Angola,1,1.0,2.0,1.0
6,Antigua and Barbuda,5,NaN,4.0,NaN
7,Argentina,5,2.0,3.0,4.0
8,Armenia,3,3.0,3.0,2.0
9,Australia,5,5.0,4.0,5.0


Un tema adicional son los valores perdidos. Hay varios NaN.

La función para reemplazarlos es sencilla, pero hay que evitar facilismos. Veamos:

In [38]:
#recordar:
DF.dtypes

Country                     object
FreedomintheWorld         category
IndexofEconomicFreedom    category
PressFreedomIndex         category
DemocracyIndex            category
dtype: object

In [39]:
#tienen que ser numericos:
DF[headers[1:]]=DF[headers[1:]].apply(pd.to_numeric)

In [40]:
#mediana por grupos: 
DF.groupby(headers[1])[headers[2:]].median()

,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
FreedomintheWorld,,,
1.0,2.0,2.0,1.0
3.0,2.0,3.0,2.0
5.0,3.0,4.0,4.0


Hemos calculado la mediana de cada indice que no sea Freedom in the world, pues ésta sólo tiene 1 valor perdido:


In [41]:
DF.info() #206 buenos de 207

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 5 columns):
Country                   207 non-null object
FreedomintheWorld         206 non-null float64
IndexofEconomicFreedom    180 non-null float64
PressFreedomIndex         189 non-null float64
DemocracyIndex            167 non-null float64
dtypes: float64(4), object(1)
memory usage: 8.2+ KB


In [42]:
# o
DF.isnull().sum() 

Country                    0
FreedomintheWorld          1
IndexofEconomicFreedom    27
PressFreedomIndex         18
DemocracyIndex            40
dtype: int64

In [43]:
import numpy as np

for h in headers[2:]:
    DF[h].fillna(DF.groupby(["FreedomintheWorld"])[h].transform(np.median), inplace=True)

Obteniendo:

In [44]:
DF.head(20)

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Abkhazia,3.0,2.0,3.0,2.0
1,Afghanistan,1.0,2.0,2.0,1.0
2,Albania,3.0,3.0,3.0,2.0
3,Algeria,1.0,1.0,2.0,1.0
4,Andorra,5.0,3.0,4.0,4.0
5,Angola,1.0,1.0,2.0,1.0
6,Antigua and Barbuda,5.0,3.0,4.0,4.0
7,Argentina,5.0,2.0,3.0,4.0
8,Armenia,3.0,3.0,3.0,2.0
9,Australia,5.0,5.0,4.0,5.0


Un detalle pequeño es enviar esta data con buenos nombres de columnas:

In [45]:
DF.columns=["Country","WorldFreedom","EconomicFreedom","PressFreedom","Democracy"]

**Guardando archivo**

A esta altura es bueno guardar el archivo, pues ya está listo:

In [33]:
DF.to_csv("indexes.csv",index=None)

Traigamos la data de los departamentos de Colombia que vimos al final de la unidad anterior:

In [34]:
idhCol2='https://es.wikipedia.org/wiki/Anexo:Departamentos_de_Colombia_por_IDH'
idhColT2=pd.read_html(idhCol2,header=0,flavor='bs4',attrs={'class': 'sortable',},
                       thousands='\xa0', decimal=',')[0]
idhColT2.iloc[:,2]=idhColT2.iloc[:,2].str.replace("\s","")
idhColT2.columns=[element.split('[')[0].replace(" ","") for element in idhColT2.columns]
idhColT2.Entidad=[element.split('[')[0] for element in idhColT2.Entidad]
newRows=['Amazonas', 'Guainia', 'Guaviare', 'Vaupés', 'Vichada']
info=idhColT2[idhColT2.Entidad=='Región Amazónica'].values.tolist()[0][1:]
newData = pd.DataFrame([[row] + info for row in newRows], columns=idhColT2.columns)
idhColT2=idhColT2.append(newData,ignore_index=True)
idhColT2.drop([0,24,29],inplace=True)
numericos=list(idhColT2.IDH)
numericos.extend(list(idhColT2.Población))

inapropiados=[]
for n in numericos:
    try:
        float(n)
    except ValueError:
        if not n in inapropiados: # evitar duplicados
            inapropiados.append(n)
idhColT2.replace(inapropiados,value=[None]*len(inapropiados),inplace=True)
idhColT2.reset_index(drop=True,inplace=True)

Así que actualmente, tenemos:

In [35]:
idhColT2

,Entidad,IDH,Población,PaísComparable
0,Santander,0.879,2340988,Hungría
1,Casanare,0.867,344027,Argentina
2,Valle del Cauca,0.865,4520166,Bahamas
3,Antioquia,0.849,6299886,Brasil
4,Boyacá,0.842,1405122,Bulgaria
5,Risaralda,0.839,941283,San Cristóbal y Nieves
6,Cundinamarca,0.837,2598245,Rumania
7,Atlántico,0.835,2403027,Montenegro
8,San Andrés,0.834,75167,Montenegro
9,Quindío,0.832,558934,Malasia


Aquí el problema es distinto. Los datos faltantes necesitamos reemplazarlos, no estimarlos. Es decir, hay que traer la data de otro sitio.

In [36]:
import IPython

pobCol='https://es.wikipedia.org/wiki/Anexo:Departamentos_de_Colombia_por_poblaci%C3%B3n'
iframe = '<iframe src=' + pobCol + ' width=700 height=350></iframe>'
IPython.display.HTML(iframe)

In [37]:

colPOB=pd.read_html(pobCol,header=0,flavor='bs4',attrs={'class': 'sortable',},
                       thousands='\xa0', decimal=',')[0]

# q ha venido?
colPOB.dtypes

N.º                   float64
Departamento           object
Población Cabecera      int64
Población Resto         int64
Población Total         int64
dtype: object

In [38]:
colPOB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 5 columns):
N.º                   33 non-null float64
Departamento          34 non-null object
Población Cabecera    34 non-null int64
Población Resto       34 non-null int64
Población Total       34 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.4+ KB


Recuerda que tenemos 32 regiones, es decir aquí ha venido algo extra: Bogotá y Colombia:

In [39]:
colPOB[colPOB.Departamento.isin (['Colombia','Bogotá','Cundinamarca'])]

,N.º,Departamento,Población Cabecera,Población Resto,Población Total
0,1.0,Bogotá,8164178,16869,8181047
3,4.0,Cundinamarca,1906623,897615,2804238
33,NaN,Colombia,38295351,11538889,49834240


De aquí, vemos que a la info de Cundinamarca debe sumarsele la de Bogotá, y eliminar luego Bogotá y Colombia.

In [40]:
colPOB.iloc[3,2:]=colPOB.iloc[3,2:]+colPOB.iloc[0,2:]

In [41]:
colPOB.drop([0,33],inplace=True)
colPOB.reset_index(drop=True,inplace=True)

In [42]:
# asi queda:
colPOB

,N.º,Departamento,Población Cabecera,Población Resto,Población Total
0,2.0,Antioquia,5262172,1428858,6691030
1,3.0,Valle del Cauca,4169553,586560,4756113
2,4.0,Cundinamarca,10070801,914484,10985285
3,5.0,Atlántico,2438533,107391,2545924
4,6.0,Bolívar,1693659,477621,2171280
5,7.0,Santander,1587972,502867,2090839
6,8.0,Nariño,907590,901526,1809116
7,9.0,Córdoba,950107,838400,1788507
8,10.0,Tolima,980694,439253,1419947
9,11.0,Cauca,567393,848540,1415933


Preparemonos para llevar esta info a la data anterior.

Eliminemos cosas innecesarias. La primera columna (No) no es necesaria:

In [43]:
colPOB.drop(['N.º'], axis=1,inplace=True)

En la data con el IDH por provincia, Población y PaísComparable tampoco son necesarias:

In [44]:
idhColT2.drop(['Población','PaísComparable'], axis=1,inplace=True)

Ambas datas tienen el mismo tamaño?

In [45]:
len(colPOB)==len(idhColT2)

True

Si es así, la unión de ambas debería ser igual, considerando lo que hemos hecho, pero siempre hay detalles que faltan:

In [46]:
test=idhColT2.merge(colPOB,left_on='Entidad',right_on='Departamento',how='outer')
test

,Entidad,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,Santander,0.879,Santander,1587972.0,502867.0,2090839.0
1,Casanare,0.867,Casanare,281548.0,93701.0,375249.0
2,Valle del Cauca,0.865,Valle del Cauca,4169553.0,586560.0,4756113.0
3,Antioquia,0.849,Antioquia,5262172.0,1428858.0,6691030.0
4,Boyacá,0.842,Boyacá,742812.0,539251.0,1282063.0
5,Risaralda,0.839,Risaralda,761658.0,206109.0,967767.0
6,Cundinamarca,0.837,Cundinamarca,10070801.0,914484.0,10985285.0
7,Atlántico,0.835,Atlántico,2438533.0,107391.0,2545924.0
8,San Andrés,0.834,NaN,NaN,NaN,NaN
9,Quindío,0.832,Quindío,506254.0,68756.0,575010.0


Nuestro test nos muestra quienes no concuerdan para el __merge__:

In [47]:
test[pd.isnull(test.IDH) | pd.isnull(test.Departamento)]

,Entidad,IDH,Departamento,Población Cabecera,Población Resto,Población Total
8,San Andrés,0.834,NaN,NaN,NaN,NaN
28,Guainia,0.768,NaN,NaN,NaN,NaN
32,NaN,NaN,San Andrés y Providencia,56487.0,21926.0,78413.0
33,NaN,NaN,Guainía,13090.0,30356.0,43446.0


Arriba se nota por que no hubo una combinación perfecta. Resolvamos y re hagamos:

In [48]:
idhColT2[idhColT2.Entidad.isin(['San Andrés', 'Guainia'])]


,Entidad,IDH
8,San Andrés,0.834
28,Guainia,0.768


In [49]:
colPOB[colPOB.Departamento.isin(['San Andrés y Providencia', 'Guainía'])]

,Departamento,Población Cabecera,Población Resto,Población Total
28,San Andrés y Providencia,56487,21926,78413
31,Guainía,13090,30356,43446


In [50]:
idhColT2.loc[8, 'Entidad']=colPOB.loc[28, 'Departamento']
idhColT2.loc[28, 'Entidad']=colPOB.loc[31, 'Departamento']

Ahora debe estar bien:

In [51]:
idhColT2.merge(colPOB,left_on='Entidad',right_on='Departamento',how='outer')

,Entidad,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,Santander,0.879,Santander,1587972,502867,2090839
1,Casanare,0.867,Casanare,281548,93701,375249
2,Valle del Cauca,0.865,Valle del Cauca,4169553,586560,4756113
3,Antioquia,0.849,Antioquia,5262172,1428858,6691030
4,Boyacá,0.842,Boyacá,742812,539251,1282063
5,Risaralda,0.839,Risaralda,761658,206109,967767
6,Cundinamarca,0.837,Cundinamarca,10070801,914484,10985285
7,Atlántico,0.835,Atlántico,2438533,107391,2545924
8,San Andrés y Providencia,0.834,San Andrés y Providencia,56487,21926,78413
9,Quindío,0.832,Quindío,506254,68756,575010


Hagamos el merge y eliminemos Entidad:

In [52]:
idhColFinal=idhColT2.merge(colPOB,left_on='Entidad',right_on='Departamento',how='outer')
idhColFinal.drop(['Entidad'],axis=1,inplace=True)
idhColFinal

,IDH,Departamento,Población Cabecera,Población Resto,Población Total
0,0.879,Santander,1587972,502867,2090839
1,0.867,Casanare,281548,93701,375249
2,0.865,Valle del Cauca,4169553,586560,4756113
3,0.849,Antioquia,5262172,1428858,6691030
4,0.842,Boyacá,742812,539251,1282063
5,0.839,Risaralda,761658,206109,967767
6,0.837,Cundinamarca,10070801,914484,10985285
7,0.835,Atlántico,2438533,107391,2545924
8,0.834,San Andrés y Providencia,56487,21926,78413
9,0.832,Quindío,506254,68756,575010


Hemos visto que el lenguaje español añade pequeñas complejidades, pues usa tildes. Creemos una columna _normalizada_ extra con el nombre del departamento:

In [53]:
#instalar unidecode
from unidecode import unidecode as notilde

In [54]:
byeTilde=lambda x: x if x is None else notilde(x)
idhColFinal[['DepartamentoNorm']]=idhColFinal[['Departamento']].applymap(byeTilde)
#
idhColFinal

,IDH,Departamento,Población Cabecera,Población Resto,Población Total,DepartamentoNorm
0,0.879,Santander,1587972,502867,2090839,Santander
1,0.867,Casanare,281548,93701,375249,Casanare
2,0.865,Valle del Cauca,4169553,586560,4756113,Valle del Cauca
3,0.849,Antioquia,5262172,1428858,6691030,Antioquia
4,0.842,Boyacá,742812,539251,1282063,Boyaca
5,0.839,Risaralda,761658,206109,967767,Risaralda
6,0.837,Cundinamarca,10070801,914484,10985285,Cundinamarca
7,0.835,Atlántico,2438533,107391,2545924,Atlantico
8,0.834,San Andrés y Providencia,56487,21926,78413,San Andres y Providencia
9,0.832,Quindío,506254,68756,575010,Quindio


Al igual que en el caso anterior, debemos a esta altura guardar nuestro archivo:

In [55]:
idhColFinal.to_csv("colombia.csv",index=None)

_____

**AUSPICIO**: 

* El desarrollo de estos contenidos ha sido posible gracias al grant del Berkeley Initiative for Transparency in the Social Sciences (BITSS) at the Center for Effective Global Action (CEGA) at the University of California, Berkeley


<center>
<img src="https://www.bitss.org/wp-content/uploads/2015/07/bitss-55a55026v1_site_icon.png" style="width: 200px;"/>
</center>

* Este curso cuenta con el auspicio de:


<center>
<img src="https://www.python.org/static/img/psf-logo@2x.png" style="width: 500px;"/>
</center>



**RECONOCIMIENTO**


EL Dr. Magallanes agradece a la Pontificia Universidad Católica del Perú, por su apoyo en la participación en la Escuela ALACIP.

<center>
<img src="https://dci.pucp.edu.pe/wp-content/uploads/2014/02/Logotipo_colores-290x145.jpg" style="width: 400px;"/>
</center>


El autor reconoce el apoyo que el eScience Institute de la Universidad de Washington le ha brindado desde el 2015 para desarrollar su investigación en Ciencia de Datos.

<center>
<img src="https://escience.washington.edu/wp-content/uploads/2015/10/eScience_Logo_HR.png" style="width: 500px;"/>
</center>

<br>
<br>